In [1]:
using Unitful
using UnitfulUS

In [2]:
# Register jupyter notebook as a source of units 
# for new units we define
Unitful.register(Main);
@unit HP "HP" Horsepower 0.7456999*u"kW" false
@unit cc "cc" CubicCentimeter 1u"cm^3" false
@unit ci "ci" CubicInch 1u"inch^3" false
@derived_dimension VolumeFlow  Unitful.𝐋^3 / Unitful.𝐓

## AFR / Lambda & Fuel Blends

TODO: store fuels so they can be accesed as `Fuels.ethanol.stoic_afr` rather than `stoic_afrs["ethanol"]`

In [3]:
const stoic_afrs = Dict([
    "methanol" => 6.47,
    "ethanol" => 9.00,
    "diesel" => 14.5,
    "gas" => 14.7
])

Dict{String, Float64} with 4 entries:
  "ethanol"  => 9.0
  "gas"      => 14.7
  "diesel"   => 14.5
  "methanol" => 6.47

In [4]:
λ(afr::Real, afr_stoic::Real) = afr / afr_stoic;
λ(afr::Real, fuel = "gas") = afr / stoic_afrs[fuel];
afr(λ::Real, fuel="gas") = λ * stoic_afrs[fuel];
afr(λ::Real, afr_stoic::Real) = λ * afr_stoic;

In [5]:
E(pct) = (pct/100) * stoic_afrs["ethanol"] + (1 - (pct/100)) * stoic_afrs["gas"];

In [6]:
λ(12.0, E(10))

0.8492569002123141

In [148]:
afr(0.8, E(10))

11.304000000000002

## Boost and Power

In [13]:
"""
Garbage calculation grabbed from: https://bit.ly/3oUG6sA
"""
function air_needs(hp_goal::Unitful.Power, 
        afr::Real, 
        bsfc)
    # mass_air_flow = air lbs / minute
    mass_air_flow = hp_goal * afr * (bsfc |> u"lbf/hr/HP") # power * air / fuel * (fuel/min) / power = 
    return mass_air_flow;
end
function boost_needs(hp_goal::Unitful.Power, 
        afr::Real, 
        bsfc, 
        charge_temp::Unitful.Temperature, 
        ve::Real, 
        max_rpm::Unitful.Frequency, 
        displacement::Unitful.Volume)
    mass_air_flow = air_needs(hp_goal, afr, bsfc) |> u"lbf/minute";
    gas_constant_r = 639.6u"inch^3/Ra/inch^2";
    charge_temp_absolute = charge_temp |> u"Ra"; # absolute Rankine/Fahrenheit
    # MAPreq = [(R*Wa*(460+Tm)]/(Vd*N/2*VE)
    # top is volume flow rate - MAF * temp correction factor using ideal gas law
    # PV/t = N/tRT - pressure * volume
    # units - lbs/min * ? * Ra = PSI*ft^3/min
    required_pressure_top = mass_air_flow * gas_constant_r * charge_temp_absolute;
    # bottom is air capacity per time - cc/min - divide by 2 revolutions per fill for 4 stroke
    required_pressure_bot = displacement * max_rpm / 2 * ve;
    required_MAP = required_pressure_top / required_pressure_bot;
    boost = required_MAP - 1u"atm";
    return boost |> u"psi";
end

boost_needs (generic function with 1 method)

In [21]:
boost_needs(150u"HP", 
    12.0, 
    0.7u"lbf/hr/HP", 
    130u"°F", 
    0.95, 
    6000u"rpm", 
    1756u"cc"
)

11.2379947811953 psi

In [20]:
air_needs(150u"HP", 12.0, 0.6u"lbf/hr/HP") |> u"lbf/minute"

18.0 lbf minute^-1